### Introduction

**Aggregation**, also referred to as **grouping**, is a technique used to condense data by categorizing it into understandable groups. Simultaneously, within these groups, you can also analyze data points using SQL-like aggregate functions (functions that you can use while summarizing data).

Imagine you operate an online shoe store. You possess data records that provide details about each customer's purchases and the corresponding dates.

In [1]:
import pandas as pd

data = {
    "customer": ["landon", "james", "zach"],
    "shoe_name": ["blue shoes", "blue shoes", "white shoes"],
    "sale_price": [5.00, 5.00, 6.00],
    "purchase_date": ["2020-10-01", "2020-10-04", "2020-10-06"]
}

df = pd.DataFrame(data)
print(df)

  customer    shoe_name  sale_price purchase_date
0   landon   blue shoes         5.0    2020-10-01
1    james   blue shoes         5.0    2020-10-04
2     zach  white shoes         6.0    2020-10-06


At the end of each month, you might want to summarize the data to see how much revenue each shoe model generated during that month. In SQL, you could achieve this with a query like:

In this example, "shoe_name" is the field we're using to group the data, and the sum of sales for each shoe model is the aggregation measure. The expected outcome would be:

This result indicates that there were two sales of blue shoes at $5 per pair and one sale of white shoes at $6 per pair. If we only used SUM(sale_price) AS revenue without the GROUP BY shoe_name, we'd only get the total revenue of $16. It wouldn't tell us which shoes were more popular.

"SUM()" is just one type of aggregate function. There are others like "MIN()", "MAX()", "COUNT()", and "STDEV()", among others. These functions help summarize distinct groups within your data.

In the upcoming exercises, we'll explore how to analyze data in groups using aggregation techniques.

#### Exercise Setup

Import relevant Spark SQL libraries using the following code:

In [2]:
import findspark
findspark.init()
findspark.find()

'C:\\Spark\\sparkhome'

In [3]:
from pyspark.sql import functions as F

Continuing to build on our animal data set (from previous entries in this course), Create a List of Rows, each containing an animal name, type, age and color using the following code:

In [4]:
# Create a list of Rows containing pet data
my_previous_pets = [("fido", "dog", 4, "brown"),
                    ("annabelle", "cat", 15, "white"),
                    ("fred", "bear", 29, "brown"),
                    ("gus", "parakeet", 2, "black"),
                    ("daisy", "cat", 8, "black"),
                    ("jerry", "cat", 1, "white"),
                    ("fred", "parrot", 1, "brown"),
                    ("gus", "fish", 1, "gold"),
                    ("gus", "dog", 11, "black"),
                    ("daisy", "iguana", 2, "green"),
                    ("rufus", "dog", 10, "gold")]

In [5]:
my_previous_pets

[('fido', 'dog', 4, 'brown'),
 ('annabelle', 'cat', 15, 'white'),
 ('fred', 'bear', 29, 'brown'),
 ('gus', 'parakeet', 2, 'black'),
 ('daisy', 'cat', 8, 'black'),
 ('jerry', 'cat', 1, 'white'),
 ('fred', 'parrot', 1, 'brown'),
 ('gus', 'fish', 1, 'gold'),
 ('gus', 'dog', 11, 'black'),
 ('daisy', 'iguana', 2, 'green'),
 ('rufus', 'dog', 10, 'gold')]

Use the parallelize() function of Spark to turn that List into an RDD, create a DataFrame from it by providing a schema, and make a temp SQL view on top of it:

In [6]:
from pyspark.sql import SparkSession

In [7]:
spark = SparkSession.builder.appName('Vamsi_App_3').getOrCreate()

In [8]:
petsRDD = spark.sparkContext.parallelize(my_previous_pets)

In [9]:
# Create a DataFrame from the RDD with a specified schema
petsDF = spark.createDataFrame(petsRDD, ['nickname', 'type', 'age', 'color'])

### Create a SQL view using the DataFrame

The **createOrReplaceTempView** function in Spark creates a temporary view (or table) that allows you to interact with a DataFrame using SQL queries. Here's what it means in more detail:

+ **Create:** It creates a temporary view based on the DataFrame you provide.

+ **Or Replace:** If a temporary view with the same name already exists, this function will replace it with the new one you're creating.

+ **Temp View:** The view you create is temporary, meaning it only exists within the context of your current Spark session and is not persisted to disk. It's meant for quick querying and analysis during your current session.

In your example, after creating the DataFrame petsDF, you use the createOrReplaceTempView function to make a temporary view named 'pets'. This view allows you to run SQL queries on the DataFrame as if it were a SQL table. This can be quite useful when you want to perform analysis using SQL syntax rather than DataFrame methods. Just remember that this view is only available for the duration of your Spark session and won't be accessible after you close the session.

In [10]:
# Create a SQL view using the DataFrame
petsDF.createOrReplaceTempView('pets')

### Analysis through Aggregation

We now have an in-memory view, similar to a relational table, of our data. This view only exists within the scope of our Spark application, but it allows us to execute SQL queries against it as if it were an actual table in a database. This feature proves to be very useful in your work with Spark.

With our queryable view in place, let's address various questions about the data using Spark's powerful SQL capabilities to query the temporary view we've set up. Once a table is registered, you can query it multiple times, treating it just like a real table.

Now, let's look at the Python solutions for the given questions:

**Q. What are the three most popular (i.e. recurring) names in the data?**

To answer this question, we need to write a SQL query that counts the occurrences of each name in the table. While you could achieve this using DataFrame methods, we'll focus on the SQL approach in this example.

In [11]:
spark.sql("select * from pets").show()

+---------+--------+---+-----+
| nickname|    type|age|color|
+---------+--------+---+-----+
|     fido|     dog|  4|brown|
|annabelle|     cat| 15|white|
|     fred|    bear| 29|brown|
|      gus|parakeet|  2|black|
|    daisy|     cat|  8|black|
|    jerry|     cat|  1|white|
|     fred|  parrot|  1|brown|
|      gus|    fish|  1| gold|
|      gus|     dog| 11|black|
|    daisy|  iguana|  2|green|
|    rufus|     dog| 10| gold|
+---------+--------+---+-----+



In [12]:
#The following shows the code in SCALA and PYTHON:
spark.sql("select nickname, count(*) as occurrences from pets group by nickname order by occurrences desc limit 3").show()

+--------+-----------+
|nickname|occurrences|
+--------+-----------+
|     gus|          3|
|    fred|          2|
|   daisy|          2|
+--------+-----------+



As can be seen the three most popular names are gus, fred, and daisy.

**Q. How old is the oldest cat in the data?**

In [13]:
spark.sql("SELECT nickname, max(age) as age FROM pets WHERE type='cat' GROUP BY nickname limit 1").show()

+---------+---+
| nickname|age|
+---------+---+
|annabelle| 15|
+---------+---+



We can determine the maximum age of cats using Spark's functional API. Although you could achieve this with pure SQL as well, this example will focus on the functional approach.

The output will provide the age of the oldest cat in the dataset.

+ Use the where() function of the DataFrame to filter the data to just cats.
+ Use the agg() function of the DataFrame to select the max age.
+ Use the show() function of the DataFrame to print the results to the console.

In code form:

In [14]:
petsDF.where("type = 'cat'")\
.agg({"age": "max"})\
.show()

+--------+
|max(age)|
+--------+
|      15|
+--------+



**Q. What are the youngest and oldest cat ages?**

+ Use the where() function of the DataFrame to filter the data to just cats.
+ Group the data by type using groupBy().
+ Then, combine the agg() function of the DataFrame with the min() and max() functions to request two metrics: min and max age. Optional: rename the columns using alias().
        
Finally, print the results to the console using the show() function of the DataFrame.

In [15]:
petsDF.where(petsDF["type"] == "cats").groupby("type").agg(F.min("age"),F.max("age")).show()

+----+--------+--------+
|type|min(age)|max(age)|
+----+--------+--------+
+----+--------+--------+



In [16]:
petsDF.where(petsDF["type"] == "cat") \
.groupBy("type") \
.agg(F.min("age"), F.max("age")) \
.show()

+----+--------+--------+
|type|min(age)|max(age)|
+----+--------+--------+
| cat|       1|      15|
+----+--------+--------+



**Q. What is the average dog age?**

+ Use the where() function of the DataFrame to filter the data to just dogs.
+ Group the data by type using groupBy().
+ Use the agg() function of the DataFrame to select the average age.

Finally, print the results to the console using the show() function of the DataFrame.

In [23]:
petsDF.where("type = 'dog'")\
.agg({"age": "avg"})\
.show()

+-----------------+
|         avg(age)|
+-----------------+
|8.333333333333334|
+-----------------+



In [24]:
petsDF.where("type = 'dog'")\
.groupBy("type")\
.agg(F.avg("age"))\
.show()

+----+-----------------+
|type|         avg(age)|
+----+-----------------+
| dog|8.333333333333334|
+----+-----------------+



**Q. How many pets of each color are there in the data?**

+ Group the data by type using groupBy().
+ Use the groupBy() function of the DataFrame to count the records in each group.

Finally, print the results to the console using the show() function of the DataFrame

In [25]:
petsDF.groupBy("color").count().show()

+-----+-----+
|color|count|
+-----+-----+
|brown|    3|
|white|    2|
|black|    3|
| gold|    2|
|green|    1|
+-----+-----+



The number of pets in each color category can be seen here.

And that’s a rough introduction to aggregation in Spark / Spark SQL! There are a lot of powerful operations you can conduct in Spark, so keep exploring the APIs!